In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.project.types.discretization import SpatialDiscretization
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid, ActiveCells, Crs
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=ActiveCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization


## Setup Time Discretization

In [3]:
from morpheus.project.types.discretization.time.TimeUnit import TimeUnit
from morpheus.project.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.project.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 2, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 3, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 4, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 5, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 6, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 7, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 8, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 9, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 10, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 11, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 12, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

In [4]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

## Setup model

In [5]:
from morpheus.project.types.Model import Model

model = Model.new()
model = model.with_updated_time_discretization(time_discretization)
model = model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

# Setup recharge boundary

In [6]:
from morpheus.project.types.boundaries.Boundary import BoundaryName, RechargeBoundary
from morpheus.project.types.boundaries.RechargeObservation import RechargeRawDataItem, RechargeRate

layers_list = [model.layers[0].layer_id]
recharge_boundary = RechargeBoundary.from_geometry(
  name=BoundaryName('recharge_boundary'),
  geometry=Polygon([[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]),
  grid=grid,
  affected_layers=[layers_list[0]],
  data=[
    RechargeRawDataItem(
      date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
      recharge_rate=RechargeRate.from_float(0.01),
    ),
    RechargeRawDataItem(
      date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
      recharge_rate=RechargeRate.from_float(0.02),
    ),
  ],
)

recharge_boundary.affected_cells = recharge_boundary.affected_cells.filter(
  lambda cell: spatial_discretization.affected_cells.contains(cell))

m.add_child(folium.GeoJson(recharge_boundary.as_geojson()))
m.add_child(folium.GeoJson(recharge_boundary.affected_cells.to_geojson(grid).as_geojson()))
for recharge_boundary.observation in recharge_boundary.observations:
  m.add_child(folium.GeoJson(recharge_boundary.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

## Calculate constant head stress periods

In [7]:
from morpheus.project.infrastructure.calculation.engines.modflow_2005.packages.RchPackageMapper import \
  calculate_rch_boundary_stress_period_data

result = calculate_rch_boundary_stress_period_data(
  spatial_discretization=spatial_discretization,
  time_discretization=time_discretization,
  layers=model.layers,
  rch_boundary=recharge_boundary,
)
result.to_dict()